## 1 | Import packages

In [7]:
import os
import h5py
import numpy as np
import pandas as pd
import tkinter
from tkinter.filedialog import askopenfilename, askopenfilenames
from collections import defaultdict
from nptdms import TdmsFile
print("done!")

done!


## 2 | Use tkinter to choose a file and get the path to it

In [2]:
# Choose a file to test the code:
root = tkinter.Tk()
root.attributes('-topmost', True) # Make window appear on top
in_path = askopenfilename() # Open dialogue to select file
print(in_path)
root.destroy() # Close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0] # Get path until folder
file_name = os.path.split(in_path)[1] # Get filename
print(folder_name)
print(file_name)

D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells/dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5
D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells
dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5


In [ ]:
def importFile():
    """
    `importFile` opens a window to select a file to import.
    Returns path and calls `openFile` to extract data."
    """

    root = tkinter.Tk()
    root.attributes('-topmost', True) # Make window appear on top
    in_path = askopenfilename() # Open dialogue to select file
    root.destroy() # Close the root window

    folder_name = os.path.split(in_path)[0] # Get path until folder
    file_name = os.path.split(in_path)[1] # Get filename
    
    voltage_mV, current_pA, command, ttl, time, dt = openFile(in_path) # Call openFile() function
    
    return voltage_mV, current_pA, command, ttl, time, dt, folder_name, file_name

## 3 | Open a file

In [ ]:
def openFile(in_path, curated_channel = None):
    """
    `openFile` checks whether you are attempting to open a `.tdms` or a `.hdf5` file.
    Extracts the data from selected channels.
    """

    if '.tdms' in in_path:
        extracted_channels, time, dt = openTDMSfile(in_path)
    elif '.hdf5' in in_path:
        extracted_channels, time, dt = openHDF5file(in_path, curated_channel = curated_channel)
    
    voltage_mV = extracted_channels[0]
    current_pA = extracted_channels[1]
    command = extracted_channels[2]
    ttl = extracted_channels[3]
    
    return voltage_mV, current_pA, command, ttl, time, dt

### 3.1 | Open an .hdf5 file and extract channels

In [12]:
# Choose a .hdf5 file to test the code
root = tkinter.Tk()
root.attributes('-topmost', True) # Make window appear on top
in_path = askopenfilename() # Open dialogue to select file
print(in_path)
root.destroy() # Close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0] # Get path until folder
file_name = os.path.split(in_path)[1] # Get filename
print(folder_name)
print(file_name)

D:/Dropbox (UCL)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells/dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5
D:/Dropbox (UCL)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells
dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5


In [ ]:
def openHDF5file(in_path,
                 channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B'],
                 curated_channel = None):
    """
    Opens the selected `.hdf5` file and extracts sorted data from chosen channels.
    
    :channel_list: list of channels to extract. If empty, defaults to 'Channel A', 'Channel B', 'Output A', 'Output B'.
    :curated_channel: e.g. copy of a 'Channel' where some sweeps/trials have been deleted due to noise or quality.
    """

    # Read hdf5 file:
    hdf5_file = h5py.File(in_path, 'r')
    
    # Define empty dictionary to populate with correctly sorted data:
    data_dict = defaultdict(list)
    # Define empty dictionary to populate with corrected trialKeys:
    key_dict = defaultdict(list)
    
    # Iterate through channels to find trial indices and sort them numerically:
    for channel in hdf5_file.keys():
        
        # Fix hdf5 indexing. Otherwise it sorts sweeps alphabetically (as 1, 10, 11, [...], 2, 21, 22...)
        if 'Channel' in channel:
            # Get keys from hdf5 (i.e. the name of each sweep/trial)
            # These have been sorted alphabetically as in strings: 
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            
            # Convert to integers so you can sort numerically:
            trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
            
            # Get the indices that will sort the array:
            trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))
            
            if curated_channel is not None:
                # Use trial keys from curated_channel to ensure same number of trials are present in all channels.
                trialKeysInHDF5 = list(hdf5_file[curated_channel].keys())
                trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
                trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))
        
        # In the case of 'Output' channels, we need to add an extra step.
        # Whereas trialKeys for "Channel" always start at "1", "Output" starts at random values like "14197".
        elif 'Output' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
            trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))
            # Transform them so they start from 1 and can be compared to the curated_channel keys:
            trialKeysInHDF5_int_from1 = [(x-min(trialKeysInHDF5_int)+1) for x in trialKeysInHDF5_int]
     
            if curated_channel is not None:
                # Compare the trial keys starting from 1 to those from the curated channel.
                # Then use the indices of matching keys to extract only the curated trials to analyse.
                trialKeysInHDF5_curated = list(hdf5_file[curated_channel].keys())
                trialKeysInHDF5_curated_int = [int(x) for x in trialKeysInHDF5_curated]
                trialKeysInHDF5_curated_sorting_indices = list(np.argsort(trialKeysInHDF5_curated_int))
                # Sort the curated integer keys so you can use them in the list.index() step.
                trialKeysInHDF5_curated_int_sorted = sorted(trialKeysInHDF5_curated_int)
                # For the sorted curated trial keys, find the index of the value matching each curated_channel trial.
                # Use this as the sorting indices.
                trialKeysInHDF5_sorting_indices = [trialKeysInHDF5_int_from1.index(trialKeysInHDF5_curated_int_sorted[i]) for i in range(len(trialKeysInHDF5_curated_int_sorted))]

        # 'Sweeps_Analysis' will be a copy of either 'Channel A' or 'Channel B' that has been curated.
        # Should be the same provided as curated_channel, which will be used to subset all the channels.
        # Won't be extracted as would only be a duplication.
        elif 'Sweeps_Analysis' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
            trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))

        # To extract 'Time':
        elif 'Time' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            trialKeysInHDF5_sorting_indices = range(len(trialKeysInHDF5))
        
        # In case there is any other channel in the hdf5 file you haven't accounted for:
        else:
            # Print a warning:
            print(f"Unrecognised {channel}: check function. This channel may not be properly sorted.")
            trialKeysInHDF5 = list(hdf5_file[curated_channel].keys())
            trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
            trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))

        # Once you have the correct indices to obtain sorted trial keys, extract the ordered data:
        for i in range(len(trialKeysInHDF5_sorting_indices)):
            correctedTrialKey = trialKeysInHDF5[trialKeysInHDF5_sorting_indices[i]] 
            data_dict[channel].append(np.array(hdf5_file[channel][correctedTrialKey]))
            key_dict[channel].append(correctedTrialKey)
    
    extracted_channels = []
    corrected_trial_keys = []

    # Keep only the useful channels and their trial keys:
    for channel in channel_list:
        extracted_channels.append(data_dict[channel])
        corrected_trial_keys.append(key_dict[channel])
    
    # Get time and delta_t
    if len(data_dict['Time']) >0:
        time = data_dict['Time']
        dt = np.mean(np.diff(time))
    else:
        dt = 0.04
        time = np.linspace(0, len(data_dict['Channel A'][0])*dt, len(['Channel A'][0]))
    
    # Create data frame of data:
    channels_data_frame = pd.DataFrame(extracted_channels, index = channel_list, columns = corrected_trial_keys[0])

    return extracted_channels, corrected_trial_keys, channel_list, channels_data_frame, time, dt

### 3.2 | Open .tdms file and extract channels

In [ ]:
# Choose a .tdms file to test the code
root = tkinter.Tk()
root.attributes('-topmost', True) # Make window appear on top
in_path = askopenfilename() # Open dialogue to select file
print(in_path)
root.destroy() # Close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0] # Get path until folder
file_name = os.path.split(in_path)[1] # Get filename
print(folder_name)
print(file_name)

In [ ]:
def openTDMSfile(in_path, channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B']):
    """
    `openTDMSfile` returns a list of arrays, where each is a sweep/trial.
    """
    
    # Load .tdms file
    tdms_file = TdmsFile(in_path) 
    data_dict = defaultdict(list)
    
    # Iterate through channels and extract data from sweeps/trials
    for group in tdms_file.groups():
        i=0
        for sweep in group.channels():
            data_dict[group.name].append(sweep.data)
            i+=1
                       
    # Keep only useful channels
    extracted_channels = []

    for channel in channel_list: 
        extracted_channels.append(data_dict[channel])
    
    # Get time and delta_t
    time = data_dict['Time'][0]
    dt = np.mean(np.diff(time))
    
    return extracted_channels, time, dt

## 4 | Find Test Pulse and obtain seal resistance

In [ ]:
def parse_TTL_edges(TTL, edgeType):
    trial =0
    edges = []
    
    while len(edges)<1 and trial < len(TTL):
    
        if 'rising' in edgeType:
            edges = np.where(np.diff(TTL[trial])>0) 
        elif 'falling' in edgeType:
            edges = np.where(np.diff(TTL[trial])<0) 
        elif 'both' in edgeType:
            edges = np.where(np.diff(TTL[trial])!=0)    
            
        edges = edges + np.full(len(edges),1) #+1 to correct the shift due to differentiation
        edges = edges[0] #just to unwrap the array
        
        trial = trial+1
    
    
    return edges